<a href="https://colab.research.google.com/github/nikolasleeb/INFO523_FinalProject/blob/main/VehicleRegistration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
import glob, os

# Attach Data

In [55]:
'''Connects to Google Drive to load in data'''

'''
# Connect to and mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Find the dataset in google drive and assign path to variable called 'Match'
matches = glob.glob('/content/drive/MyDrive/**/VehicleRegistrations.xlsx', recursive=True)
print("Found paths:", matches)

# Using 'Match' load in the dataset using the path
path = matches[0]
registrations = pd.read_excel(path, engine='openpyxl')
registrations.head()
# '''

# '''
# Load dataset from local file for demonstration purposes
registrations = pd.read_excel('VehicleRegistrations.xlsx', engine='openpyxl')
registrations.head()
# '''

,Inventory,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Number of Automobile Vehicle Registrations,136066045,1.297283e+08,129748704,131838538,132432044,133621420,137633467,135920677,135669897,...,1.112899e+08,113676345,1.138988e+08,1.128642e+08,112961266,111177029,1.112421e+08,1.085477e+08,1.051353e+08,1.029739e+08
1,Number of Motorcycle Vehicle Registrations,3767029,3.871237e+06,3826373,3879450,4152433,4346068,4903056,5004156,5370035,...,8.454939e+06,8404687,8.417718e+06,8.600936e+06,8679380,8664108,8.659741e+06,8.596314e+06,8.347435e+06,9.881414e+06
2,Motor vehicle licensed drivers,176628482,1.795393e+08,182709204,184860969,187170420,190625023,191275719,194295633,196165667,...,2.118148e+08,212159728,2.140925e+08,2.180845e+08,221994424,225346257,2.275584e+08,2.286797e+08,2.281958e+08,2.327818e+08


In [56]:
colors = { "registrations": "#DCDCAF" }

In [57]:
''' check for null and missing values'''
registrations.isnull().sum()

Inventory    0
1995         0
1996         0
1997         0
1998         0
1999         0
2000         0
2001         0
2002         0
2003         0
2004         0
2005         0
2006         0
2007         0
2008         0
2009         0
2010         0
2011         0
2012         0
2013         0
2014         0
2015         0
2016         0
2017         0
2018         0
2019         0
2020         0
2021         0
dtype: int64

# Cleaning and Transforming

In [58]:
''' Transpose the table so that years become the first column '''

id_col = registrations.columns[0]            # e.g. "Fuel economy, mpg"
registrations = registrations.round(2)
registrations = registrations.set_index(id_col).T.reset_index().rename(columns={'index':'Year'})

registrations.head()

Inventory,Year,Number of Automobile Vehicle Registrations,Number of Motorcycle Vehicle Registrations,Motor vehicle licensed drivers
0,1995,136066045.0,3767029.00,176628482.0
1,1996,129728341.0,3871237.14,179539340.0
2,1997,129748704.0,3826373.00,182709204.0
3,1998,131838538.0,3879450.00,184860969.0
4,1999,132432044.0,4152433.00,187170420.0


In [59]:
''' Verify the columns, datatypes, and number of entries'''
registrations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 4 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Year                                        27 non-null     int64  
 1   Number of Automobile Vehicle Registrations  27 non-null     float64
 2   Number of Motorcycle Vehicle Registrations  27 non-null     float64
 3   Motor vehicle licensed drivers              27 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 996.0 bytes


In [60]:
''' Data Cleaning for registrations Data '''

# Convert Year column to Year format
registrations["Year"] = pd.to_datetime(registrations["Year"], format='%Y')

# Filter to keep data where the year is between 1995 and 2021 (inclusive)
registrations = registrations[
    (registrations["Year"].dt.year >= 1995) &
    (registrations["Year"].dt.year <= 2021)
]

# Preview the first 10 rows
registrations.head(10)

Inventory,Year,Number of Automobile Vehicle Registrations,Number of Motorcycle Vehicle Registrations,Motor vehicle licensed drivers
0,1995-01-01,136066045.0,3767029.00,176628482.0
1,1996-01-01,129728341.0,3871237.14,179539340.0
2,1997-01-01,129748704.0,3826373.00,182709204.0
3,1998-01-01,131838538.0,3879450.00,184860969.0
4,1999-01-01,132432044.0,4152433.00,187170420.0
5,2000-01-01,133621420.0,4346068.00,190625023.0
6,2001-01-01,137633467.0,4903056.00,191275719.0
7,2002-01-01,135920677.0,5004156.00,194295633.0
8,2003-01-01,135669897.0,5370035.00,196165667.0
9,2004-01-01,136430651.0,5780870.00,198888912.0


In [61]:
''' Rename columns for easier access '''
registrations = registrations.rename(columns={
    'Number of Automobile Vehicle Registrations': 'Automobile_Registrations',
    'Number of Motorcycle Vehicle Registrations': 'Motorcycle_Registrations',
    'Motor vehicle licensed drivers': 'Licensed_Drivers'
})

# Summary Stats

In [62]:
registrations.describe()

Inventory,Year,Automobile_Registrations,Motorcycle_Registrations,Licensed_Drivers
count,27,2.700000e+01,2.700000e+01,2.700000e+01
mean,2008-01-01 08:53:20,1.251580e+08,6.806726e+06,2.056301e+08
min,1995-01-01 00:00:00,1.029739e+08,3.767029e+06,1.766285e+08
25%,2001-07-02 12:00:00,1.129127e+08,4.953606e+06,1.927857e+08
50%,2008-01-01 00:00:00,1.308922e+08,7.752926e+06,2.083206e+08
75%,2014-07-02 12:00:00,1.357953e+08,8.446220e+06,2.160885e+08
max,2021-01-01 00:00:00,1.376335e+08,9.881414e+06,2.327818e+08
std,NaN,1.204000e+07,1.977060e+06,1.623333e+07


In [63]:
'''' View skewness and kurtosis of consumption data '''

columns_to_analyze = registrations.columns[1:]  # Exclude the 'Year' column

for column in columns_to_analyze:
    skewness = skew(registrations[column].dropna())
    kurt = kurtosis(registrations[column].dropna())
    print(f"Column: {column}")
    print(f"  Skewness: {skewness:.4f}")
    print(f"  Kurtosis: {kurt:.4f}\n")

Column: Automobile_Registrations
  Skewness: -0.5179
  Kurtosis: -1.4172

Column: Motorcycle_Registrations
  Skewness: -0.3593
  Kurtosis: -1.4021

Column: Licensed_Drivers
  Skewness: -0.0799
  Kurtosis: -1.0196



In [64]:
''' View registrations date range and total years of data '''

print("Earliest date:", registrations["Year"].min())
print("Latest date:", registrations["Year"].max())
print("Total months of data:", len(registrations))

Earliest date: 1995-01-01 00:00:00
Latest date: 2021-01-01 00:00:00
Total months of data: 27


# Automobile Registrations

- number of new drivers

In [65]:
''' Calculate and print average automobile registrations, standard deviation, and coefficient of variation '''

mean = registrations["Automobile_Registrations"].mean()
std = registrations["Automobile_Registrations"].std()
cv = (std / mean) * 100
print(f"Average Annual Automobile Registrations: {mean:,.2f}")
print(f"Standard deviation: {std:,.2f}")
print(f"Coefficient of variation: {cv:.2f}% \n")

Average Annual Automobile Registrations: 125,157,985.01
Standard deviation: 12,040,001.98
Coefficient of variation: 9.62% 



In [66]:
''' Create a histogram to visualize the distribution of Automobile Registrations '''

# Create histogram
figvr1 = px.histogram(
    registrations,
    x="Automobile_Registrations",
    nbins=8,
    title="Distribution of Automobile Registrations",
    color_discrete_sequence=[colors["registrations"]]
)

# Customize layout
figvr1.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="Automobile Registrations",
    yaxis_title="Frequency",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=500
)

figvr1.update_traces(marker_line_color="black", marker_line_width=1, opacity=1)

figvr1.show()

In [67]:
''' Outlier Detection for Automobile Registrations using Z-Score Method and Box Plot '''

# Use an existing column name from the DataFrame (e.g., "Car")
col_name = "Automobile_Registrations"

# Compute Z-score for the selected column
registrations["Z_automobile_registrations"] = (
    (registrations[col_name] - registrations[col_name].mean())
    / registrations[col_name].std()
)

# Flag potential outliers (|Z| > 3)
registrations_outliers = registrations[np.abs(registrations["Z_automobile_registrations"]) > 3]
print(f"Detected {len(registrations_outliers)} potential outliers in '{col_name}' registrations.")
display(registrations_outliers[["Year", col_name, "Z_automobile_registrations"]])

# visual outlier detection with box plot
figvr2 = px.box(
    registrations,
    y=col_name,
    title=f"Outlier Detection: {col_name} Registrations",
    points="all",
    color_discrete_sequence=[colors["registrations"]]
)
figvr2.update_layout(template="plotly_white", title_x=0.5, height=500)
figvr2.show()

Detected 0 potential outliers in 'Automobile_Registrations' registrations.


Inventory,Year,Automobile_Registrations,Z_automobile_registrations


# Motorcycle Registrations

In [68]:
''' Calculate and print average yearly motorcycle registrations, standard deviation, and coefficient of variation '''

mean = registrations["Motorcycle_Registrations"].mean()
std = registrations["Motorcycle_Registrations"].std()
cv = (std / mean) * 100
print(f"Average annual motorcycle registrations: {mean:,.2f}")
print(f"Standard deviation: {std:,.2f}")
print(f"Coefficient of variation: {cv:.2f}% \n")

Average annual motorcycle registrations: 6,806,726.41
Standard deviation: 1,977,060.34
Coefficient of variation: 29.05% 



In [69]:
''' Create a histogram to visualize the distribution of Motorcycle Registrations '''

# Create histogram
figvr3 = px.histogram(
    registrations,
    x="Motorcycle_Registrations",
    nbins=8,
    title="Distribution of Motorcycle Registrations",
    color_discrete_sequence=[colors["registrations"]]
)

# Customize layout
figvr3.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="Motorcycle Registrations",
    yaxis_title="Frequency",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=500
)

figvr3.update_traces(marker_line_color="black", marker_line_width=1, opacity=1)

figvr3.show()

In [70]:
''' Outlier Detection for Motorcycle Registrations using Z-Score Method and Box Plot '''

# Use an existing column name from the DataFrame (e.g., "Car")
col_name = "Motorcycle_Registrations"

# Compute Z-score for the selected column
registrations["Z_motorcycle_registrations"] = (
    (registrations[col_name] - registrations[col_name].mean())
    / registrations[col_name].std()
)

# Flag potential outliers (|Z| > 3)
registrations_outliers = registrations[np.abs(registrations["Z_motorcycle_registrations"]) > 3]
print(f"Detected {len(registrations_outliers)} potential outliers in '{col_name}' registrations.")
display(registrations_outliers[["Year", col_name, "Z_motorcycle_registrations"]])

# visual outlier detection with box plot
figvr4 = px.box(
    registrations,
    y=col_name,
    title=f"Outlier Detection: {col_name}",
    points="all",
    color_discrete_sequence=[colors["registrations"]]
)
figvr4.update_layout(template="plotly_white", title_x=0.5, height=500)
figvr4.show()

Detected 0 potential outliers in 'Motorcycle_Registrations' registrations.


Inventory,Year,Motorcycle_Registrations,Z_motorcycle_registrations


# Licensed Drivers

In [71]:
''' Calculate and print average yearly licensed drivers, standard deviation, and coefficient of variation '''

mean = registrations["Licensed_Drivers"].mean()
std = registrations["Licensed_Drivers"].std()
cv = (std / mean) * 100
print(f"Average Annual licensed drivers: {mean:,.2f}")
print(f"Standard deviation: {std:,.2f}")
print(f"Coefficient of variation: {cv:.2f}% \n")

Average Annual licensed drivers: 205,630,124.95
Standard deviation: 16,233,334.02
Coefficient of variation: 7.89% 



In [72]:

''' Create a histogram to visualize the distribution of License Drivers '''

# Create histogram
figvr5 = px.histogram(
    registrations,
    x="Licensed_Drivers",
    nbins=12,
    title="Distribution of License Drivers",
    color_discrete_sequence=[colors["registrations"]]
)

# Customize layout
figvr5.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="Licensed Drivers",
    yaxis_title="Frequency",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=500
)

figvr5.update_traces(marker_line_color="black", marker_line_width=1, opacity=1)

figvr5.show()

In [73]:
''' Outlier Detection for Licensed Driver using Z-Score Method and Box Plot '''

# Use an existing column name from the DataFrame (e.g., "Car")
col_name = "Licensed_Drivers"

# Compute Z-score for the selected column
registrations["Z_licensed_drivers"] = (
    (registrations[col_name] - registrations[col_name].mean())
    / registrations[col_name].std()
)

# Flag potential outliers (|Z| > 3)
registrations_outliers = registrations[np.abs(registrations["Z_licensed_drivers"]) > 3]
print(f"Detected {len(registrations_outliers)} potential outliers in '{col_name}' registrations.")
display(registrations_outliers[["Year", col_name, "Z_licensed_drivers"]])

# visual outlier detection with box plot
figvr6 = px.box(
    registrations,
    y=col_name,
    title=f"Outlier Detection: {col_name}",
    points="all",
    color_discrete_sequence=[colors["registrations"]]
)
figvr6.update_layout(template="plotly_white", title_x=0.5, height=500)
figvr6.show()

Detected 0 potential outliers in 'Licensed_Drivers' registrations.


Inventory,Year,Licensed_Drivers,Z_licensed_drivers


# Line Graph

In [74]:
''' create a line graph showing trends over time for each vehicle type '''

fig_line = px.line(
    registrations,
    x="Year",
    y=["Automobile_Registrations", "Motorcycle_Registrations", "Licensed_Drivers"],
    title="Vehicle Registration Trends Over Time by Vehicle Type",
    labels={"value": "Number of Registrations", "variable": "Vehicle Type"},
    color_discrete_sequence=px.colors.qualitative.Set1
)
fig_line.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="Year",
    yaxis_title="Number of Registrations",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=600
)
fig_line.show()

# Export

In [75]:
# Save as CSV
registrations.to_csv("cleanedregistrations.csv", index=False)

# Optional: Save as Pickle for faster loading
registrations.to_pickle("cleanedregistrations.pkl")